The UCSC mapping seems to be missing some combinations of characters. It looks like the mapping has been trimmed to cater ont to widely used combinations. eg. "බෛ" is missing in the set for "ෛ". At the time of the development of the tool, computation capacity may have been a factor in trimming the mappings.

However, focus of this tool is to cast a reasonably wide net to capture all characters. Therefore we are generating the mappings to capture all available combinations.

Generation will adhere to the following rules:
* New mappings should include all the mappings in UCSC mappings.
* No new mapping should conflict with old mappings

In [1]:
consonants = {
    "l": "ක",
    "L": "ඛ",
    ".": "ග",
    ">": "ඝ",
    "X": "ඞ",
    "Õ": "ඟ",
    "p": "ච",
    "P": "ඡ",
    "c": "ජ",
    "CO": "ඣ",
    "®": "ඣ",
    "[": "ඤ",
    "{": "ඥ",
    "g": "ට",
    "G": "ඨ",
    "v": "ඩ",
    "V": "ඪ",
    "K": "ණ",
    "~": "ඬ",
    ";": "ත",
    ":": "ථ",
    "o": "ද",
    "O": "ධ",
    "k": "න",
    "|": "ඳ",
    "m": "ප",
    "M": "ඵ",
    "n": "බ",
    "N": "භ",
    "u": "ම",
    "U": "ඹ",
    "h": "ය",
    "r": "ර",
    ",": "ල",
    "j": "ව",
    "Y": "ශ",
    "I": "ෂ",
    "i": "ස",
    "y": "හ",
    "<": "ළ",
    "¿": "ළු",
    "*": "ෆ",
}
additional_consonants = {
    "CI": "ක්‍ෂ",
    "Cj": "ක්‍ව",
    "Ë": "ක්‍ෂ",
    "†": "ත්‍ථ",
    "…": "ත්‍ව",
    "‡": "න්‍ද",
    "JO": "න්‍ධ",
}
extra_mappings = {"`.": "ඟ", "`P": "ඦ", "`v": "ඬ", "`o": "ඳ"}
other_letters = {"„": "ද්‍ව", "`j": "ද්‍ව", "Š": "ද්‍ධ", "`O": "ද්‍ධ", "`G": "ට්‍ඨ"}
vowels = {
    "w": "අ",
    "wd": "ආ",
    "we": "ඇ",
    "wE": "ඈ",
    "b": "ඉ",
    "B": "ඊ",
    "W": "උ",
    "W!": "ඌ",
    "R": "ඍ",
    "RD": "ඎ",
    "Ì": "ඏ",
    "Ï": "ඐ",
    "t": "එ",
    "ta": "ඒ",
    "ft": "ඓ",
    "T": "ඔ",
    "´": "ඕ",
    "T!": "ඖ",
}

In [2]:
def concat(fm_prefix, fm_suffix, unicode_prefix, unicode_suffix, in_map, out_map):
    for consonant in in_map.items():
        out_map[fm_prefix + consonant[0] + fm_suffix] = (
            unicode_prefix + consonant[1] + unicode_suffix
        )


def generator(fm_prefix, fm_suffix, unicode_prefix, unicode_suffix, additional=False):

    out = {}

    concat(fm_prefix, fm_suffix, unicode_prefix, unicode_suffix, consonants, out)
    concat(fm_prefix, fm_suffix, unicode_prefix, unicode_suffix, extra_mappings, out)
    if additional:
        concat(
            fm_prefix,
            fm_suffix,
            unicode_prefix,
            unicode_suffix,
            additional_consonants,
            out,
        )
    return out

In [3]:
def printer(char_set):
    print("{", end="")
    for item in char_set.items():
        print(f"'{item[0]}': '{item[1]}'", end=", ")
    print("}", end="")

Mapping for "ෛ" seems to be missing several consonants. Due to the nature of unicode spec, modifiers needs to come after the letter. So this won't be correctly rendered for any case in the form `ff<letter>` if there is no mapping to replace it with the correct unicode form. Therefore generating the combinations and adding them.

In [4]:
char_set_list = []
set_1 = generator("ff", "", "", "ෛ")

print(set_1)

{'ffl': 'කෛ', 'ffL': 'ඛෛ', 'ff.': 'ගෛ', 'ff>': 'ඝෛ', 'ffX': 'ඞෛ', 'ffÕ': 'ඟෛ', 'ffp': 'චෛ', 'ffP': 'ඡෛ', 'ffc': 'ජෛ', 'ffCO': 'ඣෛ', 'ff®': 'ඣෛ', 'ff[': 'ඤෛ', 'ff{': 'ඥෛ', 'ffg': 'ටෛ', 'ffG': 'ඨෛ', 'ffv': 'ඩෛ', 'ffV': 'ඪෛ', 'ffK': 'ණෛ', 'ff~': 'ඬෛ', 'ff;': 'තෛ', 'ff:': 'ථෛ', 'ffo': 'දෛ', 'ffO': 'ධෛ', 'ffk': 'නෛ', 'ff|': 'ඳෛ', 'ffm': 'පෛ', 'ffM': 'ඵෛ', 'ffn': 'බෛ', 'ffN': 'භෛ', 'ffu': 'මෛ', 'ffU': 'ඹෛ', 'ffh': 'යෛ', 'ffr': 'රෛ', 'ff,': 'ලෛ', 'ffj': 'වෛ', 'ffY': 'ශෛ', 'ffI': 'ෂෛ', 'ffi': 'සෛ', 'ffy': 'හෛ', 'ff<': 'ළෛ', 'ff¿': 'ළුෛ', 'ff*': 'ෆෛ', 'ff`.': 'ඟෛ', 'ff`P': 'ඦෛ', 'ff`v': 'ඬෛ', 'ff`o': 'ඳෛ'}


Need to remove 'ළුෛ' and add special cases "එෛ" and "ත්‍රෛ"

In [5]:
set_1["fft"] = "එෛ"
set_1["ff;%"] = "ත්‍රෛ"
del set_1["ff¿"]
char_set_list.append(set_1)

Even though "ප්‍රෞ" is the only commonly used combination, no harm in generating for other consonants.

In [6]:
set_2 = generator("f", "%!", "", "්‍රෞ")
printer(set_2)

{'fl%!': 'ක්‍රෞ', 'fL%!': 'ඛ්‍රෞ', 'f.%!': 'ග්‍රෞ', 'f>%!': 'ඝ්‍රෞ', 'fX%!': 'ඞ්‍රෞ', 'fÕ%!': 'ඟ්‍රෞ', 'fp%!': 'ච්‍රෞ', 'fP%!': 'ඡ්‍රෞ', 'fc%!': 'ජ්‍රෞ', 'fCO%!': 'ඣ්‍රෞ', 'f®%!': 'ඣ්‍රෞ', 'f[%!': 'ඤ්‍රෞ', 'f{%!': 'ඥ්‍රෞ', 'fg%!': 'ට්‍රෞ', 'fG%!': 'ඨ්‍රෞ', 'fv%!': 'ඩ්‍රෞ', 'fV%!': 'ඪ්‍රෞ', 'fK%!': 'ණ්‍රෞ', 'f~%!': 'ඬ්‍රෞ', 'f;%!': 'ත්‍රෞ', 'f:%!': 'ථ්‍රෞ', 'fo%!': 'ද්‍රෞ', 'fO%!': 'ධ්‍රෞ', 'fk%!': 'න්‍රෞ', 'f|%!': 'ඳ්‍රෞ', 'fm%!': 'ප්‍රෞ', 'fM%!': 'ඵ්‍රෞ', 'fn%!': 'බ්‍රෞ', 'fN%!': 'භ්‍රෞ', 'fu%!': 'ම්‍රෞ', 'fU%!': 'ඹ්‍රෞ', 'fh%!': 'ය්‍රෞ', 'fr%!': 'ර්‍රෞ', 'f,%!': 'ල්‍රෞ', 'fj%!': 'ව්‍රෞ', 'fY%!': 'ශ්‍රෞ', 'fI%!': 'ෂ්‍රෞ', 'fi%!': 'ස්‍රෞ', 'fy%!': 'හ්‍රෞ', 'f<%!': 'ළ්‍රෞ', 'f¿%!': 'ළු්‍රෞ', 'f*%!': 'ෆ්‍රෞ', 'f`.%!': 'ඟ්‍රෞ', 'f`P%!': 'ඦ්‍රෞ', 'f`v%!': 'ඬ්‍රෞ', 'f`o%!': 'ඳ්‍රෞ', }

Need to remove obviosly incorrect ones: 2 ඣ්‍රෞ, ඤ්‍රෞ, ඥ්‍රෞ, ණ්‍රෞ, ර්‍රෞ, ල්‍රෞ, ළු්‍රෞ

In [7]:
del set_2["fCO%!"]
del set_2["f®%!"]
del set_2["f[%!"]
del set_2["f{%!"]
del set_2["fK%!"]
del set_2["fr%!"]
del set_2["f,%!"]
del set_2["f¿%!"]
char_set_list.append(set_2)

Set 3 is '්‍යෝ' eg. 'ෂ්‍යෝ'.

In [8]:
set_3 = generator("f", "Hda", "", "්‍යෝ")
printer(set_3)

{'flHda': 'ක්‍යෝ', 'fLHda': 'ඛ්‍යෝ', 'f.Hda': 'ග්‍යෝ', 'f>Hda': 'ඝ්‍යෝ', 'fXHda': 'ඞ්‍යෝ', 'fÕHda': 'ඟ්‍යෝ', 'fpHda': 'ච්‍යෝ', 'fPHda': 'ඡ්‍යෝ', 'fcHda': 'ජ්‍යෝ', 'fCOHda': 'ඣ්‍යෝ', 'f®Hda': 'ඣ්‍යෝ', 'f[Hda': 'ඤ්‍යෝ', 'f{Hda': 'ඥ්‍යෝ', 'fgHda': 'ට්‍යෝ', 'fGHda': 'ඨ්‍යෝ', 'fvHda': 'ඩ්‍යෝ', 'fVHda': 'ඪ්‍යෝ', 'fKHda': 'ණ්‍යෝ', 'f~Hda': 'ඬ්‍යෝ', 'f;Hda': 'ත්‍යෝ', 'f:Hda': 'ථ්‍යෝ', 'foHda': 'ද්‍යෝ', 'fOHda': 'ධ්‍යෝ', 'fkHda': 'න්‍යෝ', 'f|Hda': 'ඳ්‍යෝ', 'fmHda': 'ප්‍යෝ', 'fMHda': 'ඵ්‍යෝ', 'fnHda': 'බ්‍යෝ', 'fNHda': 'භ්‍යෝ', 'fuHda': 'ම්‍යෝ', 'fUHda': 'ඹ්‍යෝ', 'fhHda': 'ය්‍යෝ', 'frHda': 'ර්‍යෝ', 'f,Hda': 'ල්‍යෝ', 'fjHda': 'ව්‍යෝ', 'fYHda': 'ශ්‍යෝ', 'fIHda': 'ෂ්‍යෝ', 'fiHda': 'ස්‍යෝ', 'fyHda': 'හ්‍යෝ', 'f<Hda': 'ළ්‍යෝ', 'f¿Hda': 'ළු්‍යෝ', 'f*Hda': 'ෆ්‍යෝ', 'f`.Hda': 'ඟ්‍යෝ', 'f`PHda': 'ඦ්‍යෝ', 'f`vHda': 'ඬ්‍යෝ', 'f`oHda': 'ඳ්‍යෝ', }

Need to remove ළු්‍යෝ

In [9]:
del set_3["f¿Hda"]
char_set_list.append(set_3)

Set 4 is '්‍යො' eg. 'ෂ්‍යො'.

In [10]:
set_4 = generator("f", "Hd", "", "්‍යො")
printer(set_4)

{'flHd': 'ක්‍යො', 'fLHd': 'ඛ්‍යො', 'f.Hd': 'ග්‍යො', 'f>Hd': 'ඝ්‍යො', 'fXHd': 'ඞ්‍යො', 'fÕHd': 'ඟ්‍යො', 'fpHd': 'ච්‍යො', 'fPHd': 'ඡ්‍යො', 'fcHd': 'ජ්‍යො', 'fCOHd': 'ඣ්‍යො', 'f®Hd': 'ඣ්‍යො', 'f[Hd': 'ඤ්‍යො', 'f{Hd': 'ඥ්‍යො', 'fgHd': 'ට්‍යො', 'fGHd': 'ඨ්‍යො', 'fvHd': 'ඩ්‍යො', 'fVHd': 'ඪ්‍යො', 'fKHd': 'ණ්‍යො', 'f~Hd': 'ඬ්‍යො', 'f;Hd': 'ත්‍යො', 'f:Hd': 'ථ්‍යො', 'foHd': 'ද්‍යො', 'fOHd': 'ධ්‍යො', 'fkHd': 'න්‍යො', 'f|Hd': 'ඳ්‍යො', 'fmHd': 'ප්‍යො', 'fMHd': 'ඵ්‍යො', 'fnHd': 'බ්‍යො', 'fNHd': 'භ්‍යො', 'fuHd': 'ම්‍යො', 'fUHd': 'ඹ්‍යො', 'fhHd': 'ය්‍යො', 'frHd': 'ර්‍යො', 'f,Hd': 'ල්‍යො', 'fjHd': 'ව්‍යො', 'fYHd': 'ශ්‍යො', 'fIHd': 'ෂ්‍යො', 'fiHd': 'ස්‍යො', 'fyHd': 'හ්‍යො', 'f<Hd': 'ළ්‍යො', 'f¿Hd': 'ළු්‍යො', 'f*Hd': 'ෆ්‍යො', 'f`.Hd': 'ඟ්‍යො', 'f`PHd': 'ඦ්‍යො', 'f`vHd': 'ඬ්‍යො', 'f`oHd': 'ඳ්‍යො', }

Need to remove ළු්‍යො.

In [11]:
del set_4["f¿Hd"]
char_set_list.append(set_4)

Set 5 is '්‍යෙ' eg. 'ෂ්‍යෙ'.

In [12]:
set_5 = generator("f", "H", "", "්‍යෙ")
printer(set_5)

{'flH': 'ක්‍යෙ', 'fLH': 'ඛ්‍යෙ', 'f.H': 'ග්‍යෙ', 'f>H': 'ඝ්‍යෙ', 'fXH': 'ඞ්‍යෙ', 'fÕH': 'ඟ්‍යෙ', 'fpH': 'ච්‍යෙ', 'fPH': 'ඡ්‍යෙ', 'fcH': 'ජ්‍යෙ', 'fCOH': 'ඣ්‍යෙ', 'f®H': 'ඣ්‍යෙ', 'f[H': 'ඤ්‍යෙ', 'f{H': 'ඥ්‍යෙ', 'fgH': 'ට්‍යෙ', 'fGH': 'ඨ්‍යෙ', 'fvH': 'ඩ්‍යෙ', 'fVH': 'ඪ්‍යෙ', 'fKH': 'ණ්‍යෙ', 'f~H': 'ඬ්‍යෙ', 'f;H': 'ත්‍යෙ', 'f:H': 'ථ්‍යෙ', 'foH': 'ද්‍යෙ', 'fOH': 'ධ්‍යෙ', 'fkH': 'න්‍යෙ', 'f|H': 'ඳ්‍යෙ', 'fmH': 'ප්‍යෙ', 'fMH': 'ඵ්‍යෙ', 'fnH': 'බ්‍යෙ', 'fNH': 'භ්‍යෙ', 'fuH': 'ම්‍යෙ', 'fUH': 'ඹ්‍යෙ', 'fhH': 'ය්‍යෙ', 'frH': 'ර්‍යෙ', 'f,H': 'ල්‍යෙ', 'fjH': 'ව්‍යෙ', 'fYH': 'ශ්‍යෙ', 'fIH': 'ෂ්‍යෙ', 'fiH': 'ස්‍යෙ', 'fyH': 'හ්‍යෙ', 'f<H': 'ළ්‍යෙ', 'f¿H': 'ළු්‍යෙ', 'f*H': 'ෆ්‍යෙ', 'f`.H': 'ඟ්‍යෙ', 'f`PH': 'ඦ්‍යෙ', 'f`vH': 'ඬ්‍යෙ', 'f`oH': 'ඳ්‍යෙ', }

Need to remove ළු්‍යෙ. And adding special case "hH_": "ර්‍ය්‍ය"

In [13]:
set_5["hH_"] = "ර්‍ය්‍ය"
del set_5["f¿H"]
char_set_list.append(set_5)

Set 6 is '්‍රෝ' eg. 'ෂ්‍රෝ'.

In [14]:
set_6 = generator("f", "%da", "", "්‍රෝ")
printer(set_6)

{'fl%da': 'ක්‍රෝ', 'fL%da': 'ඛ්‍රෝ', 'f.%da': 'ග්‍රෝ', 'f>%da': 'ඝ්‍රෝ', 'fX%da': 'ඞ්‍රෝ', 'fÕ%da': 'ඟ්‍රෝ', 'fp%da': 'ච්‍රෝ', 'fP%da': 'ඡ්‍රෝ', 'fc%da': 'ජ්‍රෝ', 'fCO%da': 'ඣ්‍රෝ', 'f®%da': 'ඣ්‍රෝ', 'f[%da': 'ඤ්‍රෝ', 'f{%da': 'ඥ්‍රෝ', 'fg%da': 'ට්‍රෝ', 'fG%da': 'ඨ්‍රෝ', 'fv%da': 'ඩ්‍රෝ', 'fV%da': 'ඪ්‍රෝ', 'fK%da': 'ණ්‍රෝ', 'f~%da': 'ඬ්‍රෝ', 'f;%da': 'ත්‍රෝ', 'f:%da': 'ථ්‍රෝ', 'fo%da': 'ද්‍රෝ', 'fO%da': 'ධ්‍රෝ', 'fk%da': 'න්‍රෝ', 'f|%da': 'ඳ්‍රෝ', 'fm%da': 'ප්‍රෝ', 'fM%da': 'ඵ්‍රෝ', 'fn%da': 'බ්‍රෝ', 'fN%da': 'භ්‍රෝ', 'fu%da': 'ම්‍රෝ', 'fU%da': 'ඹ්‍රෝ', 'fh%da': 'ය්‍රෝ', 'fr%da': 'ර්‍රෝ', 'f,%da': 'ල්‍රෝ', 'fj%da': 'ව්‍රෝ', 'fY%da': 'ශ්‍රෝ', 'fI%da': 'ෂ්‍රෝ', 'fi%da': 'ස්‍රෝ', 'fy%da': 'හ්‍රෝ', 'f<%da': 'ළ්‍රෝ', 'f¿%da': 'ළු්‍රෝ', 'f*%da': 'ෆ්‍රෝ', 'f`.%da': 'ඟ්‍රෝ', 'f`P%da': 'ඦ්‍රෝ', 'f`v%da': 'ඬ්‍රෝ', 'f`o%da': 'ඳ්‍රෝ', }

Need to remove 'ණ්‍රෝ', 'න්‍රෝ', 'ය්‍රෝ', 'ර්‍රෝ', 'ල්‍රෝ', 'ළු්‍රෝ' and add "føda": "ද්‍රෝ"

In [15]:
set_6["føda"] = "ද්‍රෝ"
del set_6["fK%da"]
del set_6["fk%da"]
del set_6["fr%da"]
del set_6["f,%da"]
del set_6["f¿%da"]
char_set_list.append(set_6)

Set 7 is '්‍රො' eg. 'ෂ්‍රො'.

In [16]:
set_7 = generator("f", "%d", "", "්‍රො")
printer(set_7)

{'fl%d': 'ක්‍රො', 'fL%d': 'ඛ්‍රො', 'f.%d': 'ග්‍රො', 'f>%d': 'ඝ්‍රො', 'fX%d': 'ඞ්‍රො', 'fÕ%d': 'ඟ්‍රො', 'fp%d': 'ච්‍රො', 'fP%d': 'ඡ්‍රො', 'fc%d': 'ජ්‍රො', 'fCO%d': 'ඣ්‍රො', 'f®%d': 'ඣ්‍රො', 'f[%d': 'ඤ්‍රො', 'f{%d': 'ඥ්‍රො', 'fg%d': 'ට්‍රො', 'fG%d': 'ඨ්‍රො', 'fv%d': 'ඩ්‍රො', 'fV%d': 'ඪ්‍රො', 'fK%d': 'ණ්‍රො', 'f~%d': 'ඬ්‍රො', 'f;%d': 'ත්‍රො', 'f:%d': 'ථ්‍රො', 'fo%d': 'ද්‍රො', 'fO%d': 'ධ්‍රො', 'fk%d': 'න්‍රො', 'f|%d': 'ඳ්‍රො', 'fm%d': 'ප්‍රො', 'fM%d': 'ඵ්‍රො', 'fn%d': 'බ්‍රො', 'fN%d': 'භ්‍රො', 'fu%d': 'ම්‍රො', 'fU%d': 'ඹ්‍රො', 'fh%d': 'ය්‍රො', 'fr%d': 'ර්‍රො', 'f,%d': 'ල්‍රො', 'fj%d': 'ව්‍රො', 'fY%d': 'ශ්‍රො', 'fI%d': 'ෂ්‍රො', 'fi%d': 'ස්‍රො', 'fy%d': 'හ්‍රො', 'f<%d': 'ළ්‍රො', 'f¿%d': 'ළු්‍රො', 'f*%d': 'ෆ්‍රො', 'f`.%d': 'ඟ්‍රො', 'f`P%d': 'ඦ්‍රො', 'f`v%d': 'ඬ්‍රො', 'f`o%d': 'ඳ්‍රො', }

Need to remove 'ණ්‍රො', 'න්‍රො', 'ය්‍රො', 'ර්‍රො', 'ල්‍රො', 'ළු්‍රො' and add "fød": "ද්‍රො"

In [17]:
set_7["fød"] = "ද්‍රො"
del set_7["fK%d"]
del set_7["fk%d"]
del set_7["fh%d"]
del set_7["fr%d"]
del set_7["f,%d"]
del set_7["f¿%d"]
char_set_list.append(set_7)

Set 8 is '්‍රේ' eg. 'ෂ්‍රේ'. These mapping are paired with normalization rules derieved in prep.

In [18]:
set_8 = generator("f", "a%", "", "්‍රේ")
printer(set_8)

{'fla%': 'ක්‍රේ', 'fLa%': 'ඛ්‍රේ', 'f.a%': 'ග්‍රේ', 'f>a%': 'ඝ්‍රේ', 'fXa%': 'ඞ්‍රේ', 'fÕa%': 'ඟ්‍රේ', 'fpa%': 'ච්‍රේ', 'fPa%': 'ඡ්‍රේ', 'fca%': 'ජ්‍රේ', 'fCOa%': 'ඣ්‍රේ', 'f®a%': 'ඣ්‍රේ', 'f[a%': 'ඤ්‍රේ', 'f{a%': 'ඥ්‍රේ', 'fga%': 'ට්‍රේ', 'fGa%': 'ඨ්‍රේ', 'fva%': 'ඩ්‍රේ', 'fVa%': 'ඪ්‍රේ', 'fKa%': 'ණ්‍රේ', 'f~a%': 'ඬ්‍රේ', 'f;a%': 'ත්‍රේ', 'f:a%': 'ථ්‍රේ', 'foa%': 'ද්‍රේ', 'fOa%': 'ධ්‍රේ', 'fka%': 'න්‍රේ', 'f|a%': 'ඳ්‍රේ', 'fma%': 'ප්‍රේ', 'fMa%': 'ඵ්‍රේ', 'fna%': 'බ්‍රේ', 'fNa%': 'භ්‍රේ', 'fua%': 'ම්‍රේ', 'fUa%': 'ඹ්‍රේ', 'fha%': 'ය්‍රේ', 'fra%': 'ර්‍රේ', 'f,a%': 'ල්‍රේ', 'fja%': 'ව්‍රේ', 'fYa%': 'ශ්‍රේ', 'fIa%': 'ෂ්‍රේ', 'fia%': 'ස්‍රේ', 'fya%': 'හ්‍රේ', 'f<a%': 'ළ්‍රේ', 'f¿a%': 'ළු්‍රේ', 'f*a%': 'ෆ්‍රේ', 'f`.a%': 'ඟ්‍රේ', 'f`Pa%': 'ඦ්‍රේ', 'f`va%': 'ඬ්‍රේ', 'f`oa%': 'ඳ්‍රේ', }

Need to remove 'ණ්‍රේ', 'න්‍රේ', 'ය්‍රේ', 'ර්‍රේ', 'ල්‍රේ','ළු්‍රේ' and add "føa": "ද්‍රේ"

In [19]:
set_8["føa"] = "ද්‍රේ"
del set_8["fKa%"]
del set_8["fka%"]
del set_8["fha%"]
del set_8["fra%"]
del set_8["f,a%"]
del set_8["f¿a%"]
char_set_list.append(set_8)

Set 9 is 'ෞ' eg. 'ෂෞ'. 

In [20]:
set_9 = generator("f", "!", "", "ෞ")
printer(set_9)

{'fl!': 'කෞ', 'fL!': 'ඛෞ', 'f.!': 'ගෞ', 'f>!': 'ඝෞ', 'fX!': 'ඞෞ', 'fÕ!': 'ඟෞ', 'fp!': 'චෞ', 'fP!': 'ඡෞ', 'fc!': 'ජෞ', 'fCO!': 'ඣෞ', 'f®!': 'ඣෞ', 'f[!': 'ඤෞ', 'f{!': 'ඥෞ', 'fg!': 'ටෞ', 'fG!': 'ඨෞ', 'fv!': 'ඩෞ', 'fV!': 'ඪෞ', 'fK!': 'ණෞ', 'f~!': 'ඬෞ', 'f;!': 'තෞ', 'f:!': 'ථෞ', 'fo!': 'දෞ', 'fO!': 'ධෞ', 'fk!': 'නෞ', 'f|!': 'ඳෞ', 'fm!': 'පෞ', 'fM!': 'ඵෞ', 'fn!': 'බෞ', 'fN!': 'භෞ', 'fu!': 'මෞ', 'fU!': 'ඹෞ', 'fh!': 'යෞ', 'fr!': 'රෞ', 'f,!': 'ලෞ', 'fj!': 'වෞ', 'fY!': 'ශෞ', 'fI!': 'ෂෞ', 'fi!': 'සෞ', 'fy!': 'හෞ', 'f<!': 'ළෞ', 'f¿!': 'ළුෞ', 'f*!': 'ෆෞ', 'f`.!': 'ඟෞ', 'f`P!': 'ඦෞ', 'f`v!': 'ඬෞ', 'f`o!': 'ඳෞ', }

Need to remove 'ළුෞ'.

In [21]:
del set_9["f¿!"]
char_set_list.append(set_9)

Set 10 is 'ෝ' eg. 'ෂෝ'.

In [22]:
set_10 = generator("f", "da", "", "ෝ")
printer(set_10)

{'flda': 'කෝ', 'fLda': 'ඛෝ', 'f.da': 'ගෝ', 'f>da': 'ඝෝ', 'fXda': 'ඞෝ', 'fÕda': 'ඟෝ', 'fpda': 'චෝ', 'fPda': 'ඡෝ', 'fcda': 'ජෝ', 'fCOda': 'ඣෝ', 'f®da': 'ඣෝ', 'f[da': 'ඤෝ', 'f{da': 'ඥෝ', 'fgda': 'ටෝ', 'fGda': 'ඨෝ', 'fvda': 'ඩෝ', 'fVda': 'ඪෝ', 'fKda': 'ණෝ', 'f~da': 'ඬෝ', 'f;da': 'තෝ', 'f:da': 'ථෝ', 'foda': 'දෝ', 'fOda': 'ධෝ', 'fkda': 'නෝ', 'f|da': 'ඳෝ', 'fmda': 'පෝ', 'fMda': 'ඵෝ', 'fnda': 'බෝ', 'fNda': 'භෝ', 'fuda': 'මෝ', 'fUda': 'ඹෝ', 'fhda': 'යෝ', 'frda': 'රෝ', 'f,da': 'ලෝ', 'fjda': 'වෝ', 'fYda': 'ශෝ', 'fIda': 'ෂෝ', 'fida': 'සෝ', 'fyda': 'හෝ', 'f<da': 'ළෝ', 'f¿da': 'ළුෝ', 'f*da': 'ෆෝ', 'f`.da': 'ඟෝ', 'f`Pda': 'ඦෝ', 'f`vda': 'ඬෝ', 'f`oda': 'ඳෝ', }

Need to remove 'ළුෝ'

In [23]:
del set_10["f¿da"]
char_set_list.append(set_10)

Set 11 is 'ො' eg. 'ෂො'.

In [24]:
set_11 = generator("f", "d", "", "ො")
printer(set_11)

{'fld': 'කො', 'fLd': 'ඛො', 'f.d': 'ගො', 'f>d': 'ඝො', 'fXd': 'ඞො', 'fÕd': 'ඟො', 'fpd': 'චො', 'fPd': 'ඡො', 'fcd': 'ජො', 'fCOd': 'ඣො', 'f®d': 'ඣො', 'f[d': 'ඤො', 'f{d': 'ඥො', 'fgd': 'ටො', 'fGd': 'ඨො', 'fvd': 'ඩො', 'fVd': 'ඪො', 'fKd': 'ණො', 'f~d': 'ඬො', 'f;d': 'තො', 'f:d': 'ථො', 'fod': 'දො', 'fOd': 'ධො', 'fkd': 'නො', 'f|d': 'ඳො', 'fmd': 'පො', 'fMd': 'ඵො', 'fnd': 'බො', 'fNd': 'භො', 'fud': 'මො', 'fUd': 'ඹො', 'fhd': 'යො', 'frd': 'රො', 'f,d': 'ලො', 'fjd': 'වො', 'fYd': 'ශො', 'fId': 'ෂො', 'fid': 'සො', 'fyd': 'හො', 'f<d': 'ළො', 'f¿d': 'ළුො', 'f*d': 'ෆො', 'f`.d': 'ඟො', 'f`Pd': 'ඦො', 'f`vd': 'ඬො', 'f`od': 'ඳො', }

Need to remove 'ළුො'

In [25]:
del set_11["f¿d"]
char_set_list.append(set_11)

Set 12 is 'ේ' eg. 'ෂේ'. However, this is a special set since the round characters such as 'ම' and 'ට' have a different set of ASCII letters to represent the 'hal' forms. We'll be generating the 'hal' forms with 'a' for all characters to accommodate typing errors and the will add the round 'hal' characters separately.

In [26]:
set_12 = generator("f", "a", "", "ේ")
printer(set_12)

{'fla': 'කේ', 'fLa': 'ඛේ', 'f.a': 'ගේ', 'f>a': 'ඝේ', 'fXa': 'ඞේ', 'fÕa': 'ඟේ', 'fpa': 'චේ', 'fPa': 'ඡේ', 'fca': 'ජේ', 'fCOa': 'ඣේ', 'f®a': 'ඣේ', 'f[a': 'ඤේ', 'f{a': 'ඥේ', 'fga': 'ටේ', 'fGa': 'ඨේ', 'fva': 'ඩේ', 'fVa': 'ඪේ', 'fKa': 'ණේ', 'f~a': 'ඬේ', 'f;a': 'තේ', 'f:a': 'ථේ', 'foa': 'දේ', 'fOa': 'ධේ', 'fka': 'නේ', 'f|a': 'ඳේ', 'fma': 'පේ', 'fMa': 'ඵේ', 'fna': 'බේ', 'fNa': 'භේ', 'fua': 'මේ', 'fUa': 'ඹේ', 'fha': 'යේ', 'fra': 'රේ', 'f,a': 'ලේ', 'fja': 'වේ', 'fYa': 'ශේ', 'fIa': 'ෂේ', 'fia': 'සේ', 'fya': 'හේ', 'f<a': 'ළේ', 'f¿a': 'ළුේ', 'f*a': 'ෆේ', 'f`.a': 'ඟේ', 'f`Pa': 'ඦේ', 'f`va': 'ඬේ', 'f`oa': 'ඳේ', }

In [27]:
set_12["fÄ"] = "ඛේ"
set_12["få"] = "ඬේ"
set_12["fí"] = "බේ"
set_12["fÉ"] = "චේ"
set_12["fâ"] = "ඩේ"
set_12["fï"] = "මේ"
set_12["fÜ"] = "ටේ"
set_12["fõ"] = "වේ"
set_12["fè"] = "ධේ"
char_set_list.append(set_12)